<a href="https://colab.research.google.com/github/akvo/IWSIMS/blob/main/scripts/Rural_Water_Point_survey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Load Data

import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import folium
from folium import plugins
import numpy as np
import branca.colormap as cm
import json
import plotly.graph_objects as go
from scipy import stats
import matplotlib.colors as mcolors
import requests
import altair as alt
import warnings
import openpyxl

df=pd.read_csv('https://raw.githubusercontent.com/akvo/IWSIMS/refs/heads/main/data/Rural%20Water%20Point%20Survey.csv',encoding='utf-8',low_memory=False)

In [2]:
# @title Create Map
lat='GPS of the village/settlement? - Latitude'
lon='GPS of the village/settlement? - Longitude'
df = df[(df[lat].notna()) & (df[lon].notna())]
def rgba_to_hex(rgba):
    """Convert RGBA tuple to hex color string."""
    return mcolors.to_hex(rgba)
exclude_cols = ['_বাড়ির অবস্থান _latitude', '_বাড়ির অবস্থান _longitude','_বাড়ির অবস্থান _precision', '_বাড়ির অবস্থান _altitude',
                '_id', '_uuid', '_submission_time', '_validation_status',
                '_notes', '_status', '_submitted_by', '__version__', '_tags', '_index','start','end']
plot_columns = [col for col in df.columns if col not in exclude_cols]
output = widgets.Output()

def create_color_map(data):
    """Create a color map based on data type"""
    if pd.api.types.is_numeric_dtype(data):
        # Calculate quartiles and IQR
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1

        # Define outlier boundaries
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Create bounds using a combination of outlier boundaries and percentiles
        bounds = [
            data.min(),  # Minimum value
            lower_bound,  # Lower outlier boundary
            Q1,          # 25th percentile
            data.median(),  # Median
            Q3,          # 75th percentile
            upper_bound, # Upper outlier boundary
            data.max()   # Maximum value
        ]
        bounds = sorted(list(set([round(b, 2) for b in bounds])))  # Remove duplicates and round

        # Create colormap with more distinct colors
        colormap = cm.StepColormap(
            colors=['#313695', '#4575b4', '#74add1', '#abd9e9', '#fdae61', '#f46d43', '#d73027'],
            vmin=bounds[0],
            vmax=bounds[-1],
            index=bounds,
            caption=(f'Min: {bounds[0]:.2f} | '
                    f'Q1: {Q1:.2f} | '
                    f'Median: {data.median():.2f} | '
                    f'Q3: {Q3:.2f} | '
                    f'Max: {bounds[-1]:.2f}\n'
                    f'(IQR: {IQR:.2f})')
        )
        return colormap
    else:
        # For categorical data, create a categorical color map
        unique_values = data.value_counts()
        if len(unique_values) <= 20:
            top_categories = unique_values
        else:
            top_categories = unique_values.head(20)

        distinct_colors = [
            '#e6194B', '#3cb44b', '#4363d8', '#f58231', '#911eb4',
            '#42d4f4', '#f032e6', '#bfef45', '#fabed4', '#469990',
            '#dcbeff', '#9A6324', '#fffac8', '#800000', '#aaffc3',
            '#808000', '#ffd8b1', '#000075', '#a9a9a9', '#000000'
        ]

        color_dict = dict(zip(top_categories.index, distinct_colors[:len(top_categories)]))
        if len(unique_values) > 20:
            color_dict['Others'] = '#808080'

        return color_dict, top_categories

def update_map(column):
    # Create base map centered on mean coordinates
    with output:
      output.clear_output(wait=True)
      df_cleaned = df.dropna(subset=[column])
      m = folium.Map(
          location=[-15.002760949523436, 179.3224941777055],
          zoom_start=6,
          prefer_canvas=True
      )
      # Add GeoJSON layer
    #   style_function = lambda x: {
    #       'fillColor': 'none',
    #       'color': 'black',
    #       'weight': 1,
    #       'fillOpacity': 0
    #   }

    #   tooltip = folium.GeoJsonTooltip(
    #       fields=['tv_name'],
    #       aliases=['Village:'],
    #       style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    #   )

    #   geojson_layer = folium.GeoJson(
    #       geojson_data,
    #       name='Village Boundaries',
    #       style_function=style_function,
    #       tooltip=tooltip
    #   )
    #   geojson_layer.add_to(m)

      # Create color map
      if pd.api.types.is_numeric_dtype(df_cleaned[column]):
          color_map = create_color_map(df_cleaned[column])
          top_categories = None  # Not needed for numeric data
      else:
          color_map, top_categories = create_color_map(df_cleaned[column])

      # Add points to map
      for idx, row in df_cleaned.iterrows():
          if pd.api.types.is_numeric_dtype(df_cleaned[column]):
              color = color_map(row[column])
          else:
              # Use get() method with 'Others' as default for any category not in top 20
              color = color_map.get(row[column], '#808080')

          popup_text =f"Village : {row['Name of Village']} <br> {column} : {row[column]}"

          folium.CircleMarker(
              location=[row[lat],
                      row[lon]],
              radius=3,
              popup=popup_text,
              color=color,
              fill=True
          ).add_to(m)

      # Add color map to the map if numeric
      if pd.api.types.is_numeric_dtype(df_cleaned[column]):
          # Create a custom legend for numeric data
          legend_html = f'''
              <div style="position: fixed;
                          bottom: 50px; right: 50px;
                          width: 200px;
                          border:2px solid grey;
                          z-index:9999;
                          background-color:white;
                          padding:10px;
                          font-size:12px;">
              <p style="font-size:14px"><b>{column}</b></p>
              '''

          # Add the numeric ranges to the legend
          for i in range(len(color_map.index)-1):
              start = color_map.index[i]
              end = color_map.index[i+1]
              color = rgba_to_hex(color_map.colors[i])
              legend_html += f'<p><span style="color:{color};">●</span> {start:.2f} - {end:.2f}</p>'

          legend_html += '</div>'
          m.get_root().html.add_child(folium.Element(legend_html))
          color_map.add_to(m)
      else:
          # Add legend for categorical data
          legend_html = f'''
              <div style="position: fixed;
                          bottom: 50px; right: 50px; width: 200px; max-height: 500px;
                          border:2px solid grey; z-index:9999; background-color:white;
                          padding:10px; overflow-y: auto; font-size:12px;">
              <p style="font-size:14px"><b>{column}</b></p>
              <p style="font-size:10px">Showing top 20 categories by frequency</p>
              '''

          # Add top categories with counts
          for value, count in top_categories.items():
              color = color_map[value]
              legend_html += f'<p><span style="color:{color};">●</span> {value} ({count})</p>'

          # Add 'Others' category if there are more categories
          if len(df_cleaned[column].unique()) > 20:
              others_count = df_cleaned[column].value_counts().iloc[20:].sum()
              legend_html += f'<p><span style="color:#808080;">●</span> Others ({others_count})</p>'

          legend_html += '</div>'
          m.get_root().html.add_child(folium.Element(legend_html))

      # Add layer control
      folium.TileLayer('OpenStreetMap').add_to(m)
      folium.LayerControl().add_to(m)

      display(m)

# Create dropdown widget
dropdown = widgets.Dropdown(
    options=plot_columns,
    description='Color by:',
    style={'description_width': 'initial'}
)

# Connect the dropdown to the function
# dropdown.observe( update_map,names='value')

widgets.interactive(update_map, column=dropdown)
display(widgets.VBox([dropdown]))
display(widgets.VBox([output]))
update_map(dropdown.value)

In [4]:
# @title Create Grouped bar charts
warnings.simplefilter(action='ignore', category=FutureWarning)
groupby_dropdown = widgets.Dropdown(
    options=df.select_dtypes(include=['object']).columns.tolist(),
    description='Group By:',
)

attribute_dropdown = widgets.Dropdown(
    options=df.columns.tolist(),
    description='Attribute:',
)

# Function to update the chart
def update_chart(groupby_col, attribute_col):

    if (groupby_col and attribute_col) and  (groupby_col != attribute_col):
        if pd.api.types.is_numeric_dtype(df[attribute_col]):
            # Compute average across the groupBy
            grouped = df.groupby(groupby_col)[attribute_col].mean().reset_index()
            chart = alt.Chart(grouped).mark_bar().encode(
                x=alt.X(groupby_col, title=groupby_col),
                y=alt.Y(attribute_col, title=f'Average {attribute_col}'),
                tooltip=[groupby_col, attribute_col]
            ).properties(title=f'Average {attribute_col} by {groupby_col}')
        else:
            # Count unique values
            counts = df.groupby([groupby_col, attribute_col]).size().reset_index(name='counts')
            chart = alt.Chart(counts).mark_bar().encode(
                x=alt.X(attribute_col, title=""),
                y=alt.Y('counts', title='Counts'),
                column=groupby_col,
                color=attribute_col,  # This will create a split effect
                tooltip=[groupby_col, attribute_col, 'counts']
            ).properties(title=f'Counts of {attribute_col} by {groupby_col}')

        # Display the chart
        display(chart)

# Link the dropdowns to the update function
widgets.interactive(update_chart, groupby_col=groupby_dropdown, attribute_col=attribute_dropdown)


interactive(children=(Dropdown(description='Group By:', options=('Identifier', 'Display Name', 'Device identif…